<center><a href="https://5loi.com/about_loi"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>

# 文本分类
> Text Classification

## 情感分析 ##
> Sentimental Analysis

在本笔记本中，您将学习如何微调一个预训练模型。具体来说，我们将使用一个用于情感分析的模型。

**情感分析**是检测文本中情感的任务。我们将此问题建模为文本分类问题的一种简单形式。例如，“Gollum 的表演令人难以置信！”具有积极的情感，而“它既不浪漫也不像它应该的那样惊险。”具有消极的情感。在这种分析中，我们需要查看句子，并且我们只有两个类别：“正面”和“负面”。训练集中每个句子必须被标记为其中一个。情感分析被企业广泛用于识别客户在在线对话和反馈中对产品、品牌或服务的情感。


**目录**<br>
本笔记本涵盖以下部分：
* 数据集
    * 下载和预处理数据
    * 数据标记（可选）
* 使用预训练模型
    * 下载模型
    * 进行预测
* 微调预训练模型 


## 数据集 ##
> Dataset

在本笔记本中，我们将使用[斯坦福情感树库 (Stanford Sentiment Treebank, SST-2)](https://nlp.stanford.edu/sentiment/index.html)语料库进行情感分析。数据包含一系列句子，并带有正面和负面的二元标签。


对于文本分类，NeMo 要求数据采用特定的格式。数据需要以制表符分隔的文件（.tsv）格式存储，包含句子和标签两列。数据文件的每一行包含文本序列，其中单词之间用空格分隔，标签用[制表符]([TAB])分隔，即：`[单词] [空格] [单词] [空格] [单词] [制表符] [标签]`(`[WORD] [SPACE] [WORD] [SPACE] [WORD] [TAB] [LABEL]`)

例如：
* 
```
hide new secretions from the parental units[TAB]0
that loves its characters and communicates something rather beautiful about human nature[TAB]1
...
```


### 下载和预处理数据 ###
> Download and Preprocess Data

我们已经为您准备好了 SST-2 数据集。它应该包含三个文件：train.tsv、dev.tsv 和 test.tsv，分别用于 `训练`(`training`)、`验证`(`validation`) 和 `测试`(`test`)。

In [ ]:
import os
import wget

# set data path
DATA_DIR='data'
DATA_DIR=os.path.join(DATA_DIR, 'SST-2')

In [ ]:
# check that data folder should contain train.tsv, dev.tsv, test.tsv
!ls -l {DATA_DIR}

In [ ]:
# preview data 
print('Train:')
!head -n 5 {DATA_DIR}/train.tsv

print('Dev:')
!head -n 5 {DATA_DIR}/dev.tsv

print('Test:')
!head -n 5 {DATA_DIR}/test.tsv

`train.tsv` 和 `dev.tsv` 的格式与 NeMo 的格式非常接近，只是在文件开头多了一行标题。我们将删除这些多余的行。但是 `test.tsv` 的格式不同，并且此部分数据缺少标签。


In [ ]:
!sed 1d {DATA_DIR}/train.tsv > {DATA_DIR}/train_nemo_format.tsv
!sed 1d {DATA_DIR}/dev.tsv > {DATA_DIR}/dev_nemo_format.tsv

## 微调预训练模型 ##
> Fine-Tune a Pre-Trained Model

文本分类模型通常由一个预训练的[BERT](https://arxiv.org/pdf/1810.04805.pdf)模型和一个文本分类层组成。为了进行训练，我们可以使用配置文件来定义模型。配置文件(config)包含几个重要的部分，包括：

* **model**: 所有与模型相关的参数 - 语言模型(language model)、标记分类器(token classifier)、优化器(optimizer)和调度器(schedulers)、数据集(datasets)以及任何其他相关信息
* **trainer**: 传递给 PyTorch Lightning 的任何参数

_注意:_ NeMo 提供了一个创建配置文件的模板，建议将其作为起点，但只要遵循所需格式，您也可以创建自己的配置文件。

### 配置文件 ###
> Configuration File

In [ ]:
# define config path
MODEL_CONFIG="text_classification_config.yaml"
WORK_DIR='WORK_DIR'
os.makedirs(WORK_DIR, exist_ok=True)

In [ ]:
# download the model's configuration file 
BRANCH='main'
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

文本分类的配置文件 `text_classification_config.yaml` 指定了模型、训练和实验管理细节，例如文件位置、预训练模型和超参数。我们下载的 YAML 配置文件为大多数参数提供了默认值，但此实验必须指定一些项目。

使用 `omegaconf` 包可以更轻松地查看每个 YAML 部分，它允许您使用“点”表示法访问和操作配置键。我们将使用 `OmegaConf` 工具查看每个部分的详细信息。

In [ ]:
from omegaconf import OmegaConf

CONFIG_DIR = "/dli/task/WORK_DIR/configs"
CONFIG_FILE = "text_classification_config.yaml"

config=OmegaConf.load(CONFIG_DIR + "/" + CONFIG_FILE)

# print the entire configuration file
print(OmegaConf.to_yaml(config))

请注意，某些配置文件行，包括 `model.dataset.data_dir`，在路径位置使用 `???`，这意味着用户需要指定这些字段的值。有关模型参数的详细信息，请参阅[文档](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/text_classification.html#training-the-text-classification-model)。

首先，我们需要在配置文件中设置 `num_classes`，它指定了数据集中类的数量。对于 SST-2，我们只有两个类（0-正面和 1-负面）。所以我们将 `num_classes` 设置为 2。模型也支持超过 2 个类。(0-positive, 1-negative)

我们需要在配置文件中指定并设置 `model.train_ds.file_name`、`model.validation_ds.file_name` 和 `model.test_ds.file_name`，分别指向训练、验证和测试文件（如果存在）的路径。


In [ ]:
!ls $DATA_DIR

In [ ]:
# set num_classes to 2
config.model.dataset.num_classes=2

# set file paths
config.model.train_ds.file_path = os.path.join(DATA_DIR, 'train_nemo_format.tsv')
config.model.validation_ds.file_path = os.path.join(DATA_DIR, 'dev_nemo_format.tsv')

# You may change other params like batch size or the number of samples to be considered (-1 means all the samples)

# print the model section
print(OmegaConf.to_yaml(config.model))

In [ ]:
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# lets modify some trainer configs

# setup max number of steps to reduce training time for demonstration purposes of this tutorial
# Training stops when max_step or max_epochs is reached (earliest)
config.trainer.max_epochs = 1

# print the trainer section
print(OmegaConf.to_yaml(config.trainer))

注意：`OmegaConf.to_yaml()` 用于创建用于打印配置的正确格式。一旦 `text_classification_config.yaml` 文件加载到内存中，更改配置文件将需要重新定义 `config` 变量。

现在，我们准备初始化我们的模型。在模型初始化调用期间，将为训练和评估准备数据集和数据加载器。此外，将下载预训练的 BERT 模型，这可能需要几分钟，具体取决于所选 BERT 模型的大小。


### 下载预训练模型 ###
> Download Pre-Trained Model

在初始化模型之前，我们可能希望修改一些模型配置。例如，我们可能希望将预训练的 BERT 模型更改为另一个模型。默认模型是 `bert-base-uncased`。

In [ ]:
from nemo.collections import nlp as nemo_nlp

# complete list of supported BERT-like models
for model in nemo_nlp.modules.get_pretrained_lm_models_list(): 
    print(model)

In [ ]:
# specify the BERT-like model, you want to use
# set the `model.language_modelpretrained_model_name' parameter in the config to the model you want to use
config.model.language_model.pretrained_model_name = "bert-base-uncased"

现在，我们准备初始化我们的模型。在模型初始化调用期间，数据集和数据加载器也将为训练和验证做好准备。

此外，预训练的 BERT 模型将自动下载。请注意，第一次创建模型时，这可能需要几分钟，具体取决于所选 BERT 模型的大小。如果您的数据集很大，读取和处理所有数据集也可能需要一些时间。


In [ ]:
from nemo.collections.nlp.models import TextClassificationModel
import pytorch_lightning as pl

trainer=pl.Trainer(**config.trainer)
text_classification_model=TextClassificationModel(cfg=config.model, trainer=trainer)

### 模型训练 ###
> Model Training

In [ ]:
# start model training
trainer.fit(text_classification_model)

### 评估预测 ###
> Evaluate Predictions

对于推理，我们可以使用 `trainer.test()` 或 `model.classifytext()`。更多信息请参考[文档](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/nlp/models/text_classification/text_classification_model.py)。


In [ ]:
eval_config = OmegaConf.create({'file_path': config.model.validation_ds.file_path, 'batch_size': 64, 'shuffle': False, 'num_samples': -1})
text_classification_model.setup_test_data(test_data_config=eval_config)
trainer.test(model=text_classification_model, verbose=False)

### 推理 ###
> Inference

In [ ]:
# define the list of queries for inference
queries = ['by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .', 
           'director rob marshall went out gunning to make a great one .', 
           'uneasy mishmash of styles and genres .']
           
# max_seq_length=512 is the maximum length BERT supports.       
results = text_classification_model.classifytext(queries=queries, batch_size=3, max_seq_length=512)

print('The prediction results of some sample queries with the trained model:')
for query, result in zip(queries, results):
    print(f'Query : {query}')
    print(f'Predicted label: {result}')

In [ ]:
# restart the kernel
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**干得好！**

<center><a href="https://5loi.com/about_loi"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>